<a href="https://colab.research.google.com/github/Fuenfgeld/DMA2024TeamB/blob/main/Code/Datawarehouse_TeamB_V2_20240204.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Data Warehouse-Einrichtung für das Team B-Projekt**


---


## Projektinformationen:
- **Projektgruppe:** DMA2024TeamB
- **GitHub Repository:** [DMA2024TeamB Repository](https://github.com/Fuenfgeld/DMA2024TeamB)

## Vorbereitung:
1. **CSV-Quelldaten auf Google Drive laden:**
   - Lade die erforderlichen CSV-Dateien auf dein Google Drive hoch.

   - Verwende die folgenden Datein und behalte die Dateinamen bei:
    - [`patients.csv`](https://github.com/Fuenfgeld/DMA2024TeamB/blob/main/Daten/Quellendaten/patients.csv)
     - [`medications.csv`](https://github.com/Fuenfgeld/DMA2024TeamB/blob/main/Daten/Quellendaten/medications.csv)
     - [`conditions.csv`](https://github.com/Fuenfgeld/DMA2024TeamB/blob/main/Daten/Quellendaten/conditions.csv)
     - [`observations.csv`](https://github.com/Fuenfgeld/DMA2024TeamB/blob/main/Daten/Quellendaten/observations.csv)
     - [`encounters.csv`](https://github.com/Fuenfgeld/DMA2024TeamB/blob/main/Daten/Quellendaten/encounters.csv)

2. **Leere SQL-Datenbank erstellen:**
   - Lade eine leere SQLite-Datenbank mit dem Namen:TeamB_Warehouse.db auf Google Drive hoch
   
  - Verwende folgende Datei und behalte den Dateinamen bei:
    - [`TeamB_Warehouse.db`](https://github.com/Fuenfgeld/DMA2024TeamB/blob/main/Daten/SQL/TeamB_Warehouse.db)
   

## Durchführung der Data Warehouse-Einrichtung:
- Dieses Colab-Notebook ermöglicht die Erstellung einer Data Warehouse-Datenbank (DWH-Datenbank) aus den bereitgestellten CSV-Quelldaten. Die erstellte Datenbank dient als Quell-Datenbank für Analysen.

## Schritte im Überblick:
1. Importiere erforderliche Bibliotheken und erstelle eine Verbindung zur SQLite-Datenbank.
2. Lade die CSV-Quelldaten von Google Drive in Pandas DataFrames.
3. Erstelle Tabellen in der SQLite-Datenbank und lade die CSV-Daten in diese Tabellen.
4. Führe SQL-Befehle aus, um die Data Warehouse-Zieltablellen zu erstellen und zu befüllen.

## Ausführung:
- Führe die Schritte im Colab-Notebook nacheinander aus, um das Data Warehouse für das Team B-Projekt erfolgreich einzurichten.

**Hinweis:** Stelle sicher, dass du alle notwendigen Dateien auf Google Drive bereitgestellt und die leere SQLite-Datenbank erstellt hast, bevor du mit der Ausführung des Notebooks beginnst.


# **ETL Prozess**


---



# **Importieren von Bibliotheken:**


Die benötigten Bibliotheken, einschließlich pandas und sqlite3, werden importiert.

In [129]:
import pandas as pd
import sqlite3
from sqlite3 import Error
import random
import hashlib

# **Festlegen der Dateipfade für CSV-Dateien:**

Die Pfade zu den CSV-Dateien aus dem Synthea Datensatz werden definiert.
Die Quelldaten sollten vorher auf Google Drive abgelegt werde.



In [130]:
from google.colab import drive
drive.mount('/content/drive')



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [131]:
patients_csv = '/content/drive/MyDrive/patients.csv'
medications_csv = '/content/drive/MyDrive/medications.csv'
conditions_csv = '/content/drive/MyDrive/conditions.csv'
observations_csv = '/content/drive/MyDrive/observations.csv'
encounters_csv = '/content/drive/MyDrive/encounters.csv'

# **Erstellen einer SQLite-Datenbankverbindung:**

Eine Verbindung zur SQLite-Datenbank wird hergestellt.

In [132]:
db_path = '/content/drive/MyDrive/TeamB_Warehouse.db'
conn = sqlite3.connect(db_path)



# **Einlesen der CSV-Dateien in Pandas DataFrames:**

Die Daten aus den CSV-Dateien werden in Pandas DataFrames geladen.

In [133]:
patients_df = pd.read_csv(patients_csv)
medications_df = pd.read_csv(medications_csv)
conditions_df = pd.read_csv(conditions_csv)
observations_df = pd.read_csv(observations_csv)
encounters_df = pd.read_csv(encounters_csv)



# **Erstellen von Tabellen in der SQLite-Datenbank:**

Die Tabellen *patients*, *medications*, *conditions*, *observations* und *encounters* werden als separate Tabellen in der SQLite-Datenbank erstellt.

In [134]:
patients_df.to_sql('patients', conn, index=False, if_exists='replace')
medications_df.to_sql('medications', conn, index=False, if_exists='replace')
conditions_df.to_sql('conditions', conn, index=False, if_exists='replace')
observations_df.to_sql('observations', conn, index=False, if_exists='replace')
encounters_df.to_sql('encounters', conn, index=False, if_exists='replace')



8603

# **Ausführen von SQL-Befehlen zum Erstellen und Befüllen der Zieltabelle:**

SQL-Befehle werden verwendet, um eine leere F_BreastCancerQoL-Fakten-Tabelle zu erstellen.
Temporäre Tabellen werden erstellt, um bestimmte Informationen zu extrahieren.
Diverse Dimensionstabellen (d_Patients, d_Agegroups, d_Medications, d_Cancertherapy, d_Comorbidity, d_PrimaryTumor) werden erstellt.
Die F_BreastCancerQoL-Tabelle wird mit den relevanten Informationen aus den temporären und dimensionalen Tabellen gefüllt.


Im letzten Schritt werden temporäre Tabellen und nicht mehr benötigte Originaltabellen  gelöscht.

In [135]:
sql_commands = """
-- Leere FACT-Tabelle erstellen
DROP TABLE IF EXISTS F_BreastCancerQoL;

CREATE TABLE F_BreastCancerQoL (
  PatientId UUID,
  Age_group_diagnosis_ID Integer,
  Primary_Tumor_ID String,
  Comorbidity_ID String,
  Comorbidity_YesNo Integer,
  Ethnicity String,
  Gender String,
  Cancertherapy_ID String,
  Chemotherapy_YesNo Integer,
  Targetedtherapy_YesNo Integer,
  Medication_ID String,
  Medication_YesNo Integer,
  QoL_Value_first Integer,
  QoL_Value_last Integer
);

-- Temporäre Tabelle mit MedicationID (=Medication.Code) erstellen (ohne Krebstherapie):
DROP TABLE IF EXISTS temp_patients_medicationID;

CREATE TABLE temp_patients_medicationID AS
SELECT DISTINCT p.id, m.code
FROM patients p
JOIN medications m ON p.id = m.patient
JOIN conditions c ON p.id = c.patient
WHERE m.reasoncode != '254837009' AND c.code = '254837009';

-- Temporäre Tabelle mit CancertherapieID (= Medication.Code):
DROP TABLE IF EXISTS temp_patients_cancertherapyID;

CREATE TABLE temp_patients_cancertherapyID AS
SELECT DISTINCT p.id, m.code
FROM patients p
JOIN medications m ON p.id = m.patient
JOIN conditions c ON p.id = c.patient
WHERE m.reasoncode = '254837009' AND c.code = '254837009';

-- Temporäre Tabelle mit PatientID erstellen (Nur Brustkrebspatienten):
DROP TABLE IF EXISTS temp_patients;

CREATE TABLE temp_patients AS
SELECT DISTINCT p.id, p.ethnicity, p.gender
FROM patients p
JOIN observations o ON p.id = o.patient
JOIN conditions c ON p.id = c.patient
WHERE c.code = '254837009';

-- Temporäre Tabelle mit Comorbidities erstellen (ohne Brustkrebserkrankungen):
DROP TABLE IF EXISTS temp_patients_comorbidity;

CREATE TABLE temp_patients_comorbidity AS
SELECT DISTINCT p.id, c.code
FROM conditions c
JOIN temp_patients p ON p.id = c.patient
WHERE c.code != '254837009';

-- Temporäre Tabelle mit erstem QoL Wert erstellen
DROP TABLE IF EXISTS temp_qol_value_first;

CREATE TABLE temp_qol_value_first AS
SELECT o.value AS qol_value_first, c.patient, MIN(o.date) AS earliest_date
FROM observations o
JOIN conditions c ON o.PATIENT = c.patient
WHERE o.code = 'QOLS' AND c.code = '254837009'
GROUP BY c.patient
ORDER BY c.patient;

-- Temporäre Tabelle mit letztem QoL Wert erstellen:
DROP TABLE IF EXISTS temp_qol_value_last;

CREATE TABLE temp_qol_value_last AS
SELECT o.value AS qol_value_last, c.patient, MAX(o.date) AS latest_date
FROM observations o
JOIN conditions c ON o.PATIENT = c.patient
WHERE o.code = 'QOLS' AND c.code = '254837009'
GROUP BY c.patient
ORDER BY c.patient;

-- d_Patients Tabelle erstellen:
DROP TABLE IF EXISTS d_Patients;

CREATE TABLE d_Patients AS
SELECT
  p.id AS Id,
  STRFTIME('%Y', 'now') - STRFTIME('%Y', p.birthdate) - (STRFTIME('%m-%d', 'now') < STRFTIME('%m-%d', p.birthdate)) AS AGE_Studyinclusion,
  STRFTIME('%Y', c.START) - STRFTIME('%Y', p.birthdate) - (STRFTIME('%m-%d', c.START) < STRFTIME('%m-%d', p.birthdate)) AS AGE_diagnosis,
  p.marital AS Maritalstatus
FROM
  patients p
JOIN conditions c ON p.id = c.patient
WHERE c.code = '254837009';

-- d_Agegroups Tabelle erstellen:
DROP TABLE IF EXISTS d_Agegroups;

CREATE TABLE d_Agegroups (
  ID Integer [primary key],
  Age_group String [not null],
  Min_Age Integer [not null],
  Max_Age Integer [not null]
);

INSERT INTO d_Agegroups VALUES
(1, '<50', 0, 49),
(2, '>=50', 50, 999);


-- d_Medications Tabelle erstellen:
DROP TABLE IF EXISTS d_Medications;

CREATE TABLE d_Medications AS
SELECT DISTINCT
  code AS ID,
  CASE
    WHEN Description LIKE '%Clopidogrel%' THEN 'Clopidogrel'
    WHEN Description LIKE '%Simvastatin%' THEN 'Simvastatin'
    WHEN Description LIKE '%Amlodipine%' THEN 'Amlodipine'
    WHEN Description LIKE '%Nitroglycerin%' THEN 'Nitroglycerin'
    WHEN Description LIKE '%Warfarin%' THEN 'Warfarin'
    WHEN Description LIKE '%Verapamil%' THEN 'Verapamil'
    WHEN Description LIKE '%Digoxin%' THEN 'Digoxin'
    WHEN Description LIKE '%Epinephrine%' THEN 'Epinephrine'
    WHEN Description LIKE '%Amiodarone%' THEN 'Amiodarone'
    WHEN Description LIKE '%Alteplase%' THEN 'Alteplase'
    WHEN Description LIKE '%Atropine%' THEN 'Atropine'
    WHEN Description LIKE '%Atorvastatin%' THEN 'Atorvastatin'
    WHEN Description LIKE '%Captopril%' THEN 'Captopril'
  END AS Description
FROM medications
WHERE Description LIKE '%Clopidogrel%'
   OR Description LIKE '%Simvastatin%'
   OR Description LIKE '%Amlodipine%'
   OR Description LIKE '%Nitroglycerin%'
   OR Description LIKE '%Warfarin%'
   OR Description LIKE '%Verapamil%'
   OR Description LIKE '%Digoxin%'
   OR Description LIKE '%Epinephrine%'
   OR Description LIKE '%Amiodarone%'
   OR Description LIKE '%Alteplase%'
   OR Description LIKE '%Atropine%'
   OR Description LIKE '%Atorvastatin%'
   OR Description LIKE '%Captopril%';

-- d_Cancertherapy Tabelle erstellen:
DROP TABLE IF EXISTS d_Cancertherapy;

CREATE TABLE d_Cancertherapy AS
SELECT DISTINCT
  code AS ID,
  CASE
    WHEN Description LIKE '%Anastrozole%' THEN 'Anastrozole (AI)'
    WHEN Description LIKE '%Trastuzumab%' THEN 'Trastuzumab (anti-HER2 antibody)'
    WHEN Description LIKE '%Palbociclib%' THEN 'Palbociclib (CDK4/6 inhibitor)'
    WHEN Description LIKE '%Tamoxifen%' THEN 'Tamoxifen (SERM)'
    WHEN Description LIKE '%Verzenio%' THEN 'Abemaciclib (CDK4/6 inhibitor)'
    WHEN Description LIKE '%Epirubicin%' THEN 'Epirubicin'
    WHEN Description LIKE '%Cyclophosphamide%' THEN 'Cyclophosphamide'
    WHEN Description LIKE '%Paclitaxel%' THEN 'Paclitaxel'
  END AS Description,
  CASE
    WHEN Description LIKE '%Epirubicin%' THEN 1
    WHEN Description LIKE '%Cyclophosphamide%' THEN 1
    WHEN Description LIKE '%Paclitaxel%' THEN 1
    ELSE 0
  END AS Chemotherapy,
  CASE
    WHEN Description LIKE '%Anastrozole%' THEN 1
    WHEN Description LIKE '%Trastuzumab%' THEN 1
    WHEN Description LIKE '%Palbociclib%' THEN 1
    WHEN Description LIKE '%Tamoxifen%' THEN 1
    WHEN Description LIKE '%Verzenio%' THEN 1
    ELSE 0
  END AS TargetedTherapy
FROM medications
WHERE Description LIKE '%Anastrozole%'
   OR Description LIKE '%Trastuzumab%'
   OR Description LIKE '%Palbociclib%'
   OR Description LIKE '%Tamoxifen%'
   OR Description LIKE '%Verzenio%'
   OR Description LIKE '%Epirubicin%'
   OR Description LIKE '%Cyclophosphamide%'
   OR Description LIKE '%Paclitaxel%';

--d_Comorbidity erstellen:
DROP TABLE IF EXISTS d_Comorbidity;

CREATE TABLE d_Comorbidity AS
SELECT DISTINCT code AS ID, description AS Description
FROM conditions
WHERE code != '254837009';

--d_PrimaryTumor erstellen:
DROP TABLE IF EXISTS d_PrimaryTumor;

CREATE TABLE d_PrimaryTumor AS
SELECT
  o.encounter AS ID,
  MAX(CASE WHEN o.code = '85319-2' THEN o.value END) AS HER2_Status,
  MAX(CASE WHEN o.code = '10480-2' THEN o.value END) AS HR_Status,
  MAX(CASE WHEN o.code = '21908-9' THEN o.value END) AS Stage,
  MAX(CASE WHEN o.code = '21906-3' THEN o.value END) AS Nodal_Status,
  MAX(CASE WHEN o.code = '21905-5' THEN o.value END) AS T_Status,
  MAX(CASE WHEN o.code = '21907-1' THEN o.value END) AS M_Status
FROM
  observations o
WHERE
  o.code IN ('85319-2', '10480-2', '21908-9', '21906-3', '21905-5', '21907-1')
GROUP BY
  o.encounter;

-- Temporäre Tabelle mit Primary Tumor ID erstellen (nur wenn Histologische Info vorhanden):
DROP TABLE IF EXISTS temp_encounter_primary_tumor;

CREATE TABLE temp_encounter_primary_tumor AS
SELECT e.patient, pt.id AS primarytumor_ID from encounters e, d_PrimaryTumor pt
WHERE e.id = pt.id;

-- Fülle F_BreastCancerQoL auf:

INSERT INTO F_BreastCancerQoL (PatientId, Age_group_diagnosis_ID, primary_tumor_id, Comorbidity_ID, Comorbidity_YesNo, Ethnicity, Gender, Cancertherapy_ID, Chemotherapy_YesNo, Targetedtherapy_YesNo, Medication_ID, Medication_YesNo, QoL_Value_first, QoL_Value_last)
SELECT
  p.id AS PatientId,
  CASE
    WHEN d.Age_diagnosis < 50 THEN 1
    WHEN d.age_diagnosis >= 50 THEN 2
    ELSE NULL
  END AS Age_group_diagnosis_ID,
  pt.primarytumor_ID AS primary_tumor_id,
  c.code AS Comorbidity_ID,
  CASE WHEN c.code IS NOT NULL THEN 1 ELSE 0 END AS Comorbidity_YesNo,
  p.ethnicity AS Ethnicity,
  p.gender AS Gender,
  ct.code AS Cancertherapy_ID,
  dct.chemotherapy AS Chemotherapy_YesNo,
  dct.targetedtherapy AS Targetedtherapy_YesNo,
  m.code AS Medication_ID,
  CASE WHEN m.code IS NOT NULL THEN 1 ELSE 0 END AS Medication_YesNo,
  qvf.qol_value_first AS QoL_Value_first,
  qvl.qol_value_last AS QoL_Value_last
FROM d_patients d
LEFT JOIN temp_patients_comorbidity c ON d.id = c.id
LEFT JOIN temp_patients_cancertherapyID ct ON d.id = ct.id
LEFT JOIN temp_patients_medicationID m ON d.id = m.id
LEFT JOIN temp_qol_value_first qvf ON d.id = qvf.patient
LEFT JOIN temp_qol_value_last qvl ON d.id = qvl.patient
LEFT JOIN temp_patients p ON d.id = p.id
LEFT JOIN d_Cancertherapy dct ON ct.code = dct.ID
LEFT JOIN temp_encounter_primary_tumor pt ON p.id = pt.patient;

-- Drop temp tables

DROP TABLE IF EXISTS temp_patients;
DROP TABLE IF EXISTS temp_patients_cancertherapyID;
DROP TABLE IF EXISTS temp_patients_medicationID;
DROP TABLE IF EXISTS temp_patients_comorbidity;
DROP TABLE IF EXISTS temp_qol_value_first;
DROP TABLE IF EXISTS temp_qol_value_last;
DROP TABLE IF EXISTS temp_encounter_primary_tumor;

-- Drop other tables

DROP TABLE IF EXISTS medications;
DROP TABLE IF EXISTS conditions;
DROP TABLE IF EXISTS observations;
DROP TABLE IF EXISTS careplans;
DROP TABLE IF EXISTS conditions;
DROP TABLE IF EXISTS devices;
DROP TABLE IF EXISTS disease;
DROP TABLE IF EXISTS encounters;
DROP TABLE IF EXISTS imaging_studies;
DROP TABLE IF EXISTS immunizations;
DROP TABLE IF EXISTS patients;
DROP TABLE IF EXISTS procedures;
"""
conn.executescript(sql_commands)



# **Anonymisierung der Patienten IDs:**

In [136]:
# Verbindung zur DWH-DB herstellen
conn = sqlite3.connect(db_path)
if conn is not None:
    cursor = conn.cursor()
else:
    print("Verbindung fehlgeschlagen. Bitte überprüfen!")

# Alle Id's aus der Tabelle "d_Patients" ziehen
cursor.execute("SELECT Id from d_Patients;")
d_Patients_ids = cursor.fetchall()

# Für spätere Tests die Anzahl der Patienten-Ids in der Tabelle F_BreastCancerQoL auslesen
cursor.execute("SELECT PatientId from F_BreastCancerQoL;")
before_hash_pid_F_BreastCancerQoL = cursor.fetchall()
before_hash_pid_F_BreastCancerQoL = len(before_hash_pid_F_BreastCancerQoL)

# Check, müssen 11 Patienten-Ids sein
print("Anzahl der Patienten Ids (muss 11 sein): " + str(len(d_Patients_ids)))

# Dictionary erstellen
anonymized_Patients_ids = {}
for ids in d_Patients_ids:
    # Eigentliche Anonymisierung, hashen der bisherigen ID mittels SHA256-Verschlüsselung,
    # als Salt wird eine randomisierte Zeichenkette verwendet (random.random)
    anonymized_patients_ids[ids[0]] = hashlib.sha256(f"{ids[0]}={random.random()}".encode()).hexdigest()

# Check, ob alle Ids erfolgreich gehasht wurden
# Wenn "alte" ID gleich "neuer" ID, dann Fehlermeldung
for id in d_Patients_ids:
    if id[0] in anonymized_patients_ids.values():
        print("Eine oder mehrere IDs wurden nicht gehasht!")

# Änderungen in die DB-Tabellen "d_Patients" und "F_BreastCancerQoL" schreiben, dazu Tabellen, welche geändert werden sollen, jeweils aus DB ziehen
d_Patients_table_df = pd.read_sql_query("SELECT * FROM d_Patients", conn)
F_BreastCancerQoL_df = pd.read_sql_query("SELECT * FROM F_BreastCancerQoL", conn)

# Ersetzen der Ids in der Tabelle d_Patients, Spalte "Id"
d_Patients_table_df = d_Patients_table_df.replace({"Id": anonymized_patients_ids})
# Berechnung von Hashes und einer Checksumme für die Tabelle d_Patients
d_Patients_table_df_noid = d_Patients_table_df.drop(columns=["Id"])
d_Patients_hashes = hash_pandas_object(d_patients_table_df_noid)
print("Checksumme d_Patients: ", d_Patients_hashes.sum())
if (d_Patients_hashes.sum() == -7487642478202879280):
    print("Die Überprüfung der pseudonymisierten Daten in d_Patients per Checksumme war erfolgreich.\n")
else:
    print("Die Überprüfung der pseudonymisierten Daten in d_Patients per Checksumme war nicht erfolgreich.\n")

# Ersetzen der Ids in der Tabelle F_BreastCancerQoL, Spalte "PatientID"
F_BreastCancerQoL_df = F_BreastCancerQoL_df.replace({"PatientId": anonymized_patients_ids})
# Berechnung von Hashes und einer Checksumme für die Tabelle F_BreastCancerQoL
F_BreastCancerQoL_df_noid = F_BreastCancerQoL_df.drop(columns=["PatientId"])
F_BreastCancerQoL_hashes = hash_pandas_object(F_BreastCancerQoL_df_noid)
print("Checksumme F_BreastCancerQoL: ", F_BreastCancerQoL_hashes.sum())
if (F_BreastCancerQoL_hashes.sum() == 3629088998158101975):
    print("Die Überprüfung der pseudonymisierten Daten in F_BreastCancerQoL per Checksumme war erfolgreich.\n")
else:
    print("Die Überprüfung der pseudonymisierten Daten in F_BreastCancerQoL per Checksumme war nicht erfolgreich.\n")

# Bisherige Tabellen in der DB löschen
conn.execute("DROP TABLE IF EXISTS d_Patients;")
conn.execute("DROP TABLE IF EXISTS F_BreastCancerQoL;")

# Veränderte, Anonymisierte Tabellen in die DB schreiben
d_Patients_table_df.to_sql(name="d_Patients", con=conn, if_exists='replace', index=False)
F_BreastCancerQoL_df.to_sql(name="F_BreastCancerQoL", con=conn, if_exists='replace', index=False)

# Zur Sicherheit: Alle Tabellennamen aus der Datenbank ziehen...
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tablelist = cursor.fetchall()

# ...und die Tabellennamen ausgeben
if tablelist == []:
    print("In der Data Warehouse Datenbank sind keine Tabellen vorhanden. Sie können mit der Ausführung des Skriptes fortfahren.")
else:
    print("Folgende Tabellen sind in der Datenbank vorhanden: ")
    print(tablelist)

# Testen, ob die gehashten Werte in der DB stehen
cursor.execute("SELECT Id from d_Patients;")
hashtest_patients = cursor.fetchall()
cursor.execute("SELECT PatientId from F_BreastCancerQoL;")
hashtest_encounters = cursor.fetchall()

print("\n" + "Hashtabellen von d_Patients und F_BreastCancerQoL:")
print(hashtest_patients)
print(hashtest_encounters)

# Testen, ob die Übertragung mittels Dictionary funktioniert hat:
amount_ids = 0
for ids_total in hashtest_patients:
    if ids_total in hashtest_encounters:
        amount_ids = amount_ids + 1

# Erhält man als Ergebnis 11, so sollte der Vorgang funktioniert haben
if amount_ids == 11:
    print("\n" + "Die ID aller 11 Patienten in der Tabelle d_Patients wurde gehasht.")

# Final überprüfen, ob vor und nach dem Hashvorgang die gleiche Anzahl an Ids in der Tabelle F_BreastCancerQoL stehen:
cursor.execute("SELECT PatientId from F_BreastCancerQoL;")
after_hash_pid_F_BreastCancerQoL = cursor.fetchall()
after_hash_pid_F_BreastCancerQoL = len(after_hash_pid_F_BreastCancerQoL)

if before_hash_pid_F_BreastCancerQoL == after_hash_pid_F_BreastCancerQoL:
    print("Alle Patient_IDs in der Tabelle F_BreastCancerQoL wurden erfolgreich gehasht.")
else:
    print("Es wurden nicht alle Patient_IDs in der F_BreastCancerQoL gehasht.")

Anzahl der Patienten Ids (muss 11 sein): 11
Checksumme d_Patients:  -7487642478202879280
Die Überprüfung der pseudonymisierten Daten in d_Patients per Checksumme war erfolgreich.

Checksumme F_BreastCancerQoL:  -1342178853738958170
Die Überprüfung der pseudonymisierten Daten in F_BreastCancerQoL per Checksumme war nicht erfolgreich.

Folgende Tabellen sind in der Datenbank vorhanden: 
[('d_Agegroups',), ('d_Medications',), ('d_Cancertherapy',), ('d_Comorbidity',), ('d_PrimaryTumor',), ('d_Patients',), ('F_BreastCancerQoL',)]

Hashtabellen von d_Patients und F_BreastCancerQoL:
[('d327bc941b7432b4ce67673374d65779059f0c2d0d2f1f938142422e1a07e718',), ('d8b8f7a3c33e82945070be48c82e7d42c397db128e375f03304f145a5dade24b',), ('b4d14b001866aa224846236271aeaee7e56a39aabcc90eaf3f8c52348a4a0ec6',), ('6a165118b4d3d1fb30d0e6295c8ce2551dc5abe193d763b40f6c1c814c347b68',), ('a55c3b9d9cb8b3ce64293b4ea89363f29dc8a13fbefccddf7e9de0f199dc0a44',), ('50da868f38af778f1dd5408048fd1e9539baabedeaccfb75521a80033f1

# **Anonymisierung der PrimaryTumor IDs:**

In [137]:
# Verbindung zur DWH-DB herstellen
conn = sqlite3.connect(db_path)
if conn is not None:
    cursor = conn.cursor()
else:
    print("Verbindung fehlgeschlagen. Bitte überprüfen!")

# Alle Id's aus der Tabelle "d_PrimaryTumor" ziehen
cursor.execute("SELECT ID from d_PrimaryTumor;")
d_primary_tumor_ids = cursor.fetchall()

# Für spätere Tests die Anzahl der Primary_Tumor_IDs in der Tabelle F_BreastCancerQoL auslesen
cursor.execute("SELECT Primary_Tumor_ID from F_BreastCancerQoL;")
before_hash_primary_tumor_F_BreastCancerQoL = cursor.fetchall()
before_hash_primary_tumor_F_BreastCancerQoL = len(before_hash_primary_tumor_F_BreastCancerQoL)

# Check, müssen 11 Primary_Tumor_IDs sein
print("Anzahl der Primary_Tumor_IDs (muss 11 sein): " + str(len(d_primary_tumor_ids)))

# Dictionary erstellen
anonymized_primary_tumor_ids = {}
for primary_tumor_id in d_primary_tumor_ids:
    # Eigentliche Anonymisierung, hashen der bisherigen ID mittels SHA256-Verschlüsselung,
    # als Salt wird eine randomisierte Zeichenkette verwendet (random.random)
    anonymized_primary_tumor_ids[primary_tumor_id[0]] = hashlib.sha256(f"{primary_tumor_id[0]}={random.random()}".encode()).hexdigest()

# Check, ob alle Primary_Tumor_IDs erfolgreich gehasht wurden
# Wenn "alte" ID gleich "neuer" ID, dann Fehlermeldung
for primary_tumor_id in d_primary_tumor_ids:
    if primary_tumor_id[0] in anonymized_primary_tumor_ids.values():
        print("Eine oder mehrere IDs wurden nicht gehasht!")

# Änderungen in die DB-Tabellen "d_PrimaryTumor" und "F_BreastCancerQoL" schreiben, dazu Tabellen, welche geändert werden sollen, jeweils aus DB ziehen
d_primary_tumor_table_df = pd.read_sql_query("SELECT * FROM d_PrimaryTumor", conn)
F_BreastCancerQoL_df = pd.read_sql_query("SELECT * FROM F_BreastCancerQoL", conn)

# Ersetzen der Ids in der Tabelle d_PrimaryTumor, Spalte "ID"
d_primary_tumor_table_df = d_primary_tumor_table_df.replace({"ID": anonymized_primary_tumor_ids})
# Berechnung von Hashes und einer Checksumme für die Tabelle d_PrimaryTumor
d_primary_tumor_table_df_noid = d_primary_tumor_table_df.drop(columns=["ID"])
d_primary_tumor_hashes = hash_pandas_object(d_primary_tumor_table_df_noid)
print("Checksumme d_PrimaryTumor: ", d_primary_tumor_hashes.sum())
if (d_primary_tumor_hashes.sum() == 842637750972571563):
    print("Die Überprüfung der pseudonymisierten Daten in d_PrimaryTumor per Checksumme war erfolgreich.\n")
else:
    print("Die Überprüfung der pseudonymisierten Daten in d_PrimaryTumor per Checksumme war nicht erfolgreich.\n")

# Ersetzen der Ids in der Tabelle F_BreastCancerQoL, Spalte "Primary_Tumor_ID"
F_BreastCancerQoL_df = F_BreastCancerQoL_df.replace({"Primary_Tumor_ID": anonymized_primary_tumor_ids})
# Berechnung von Hashes und einer Checksumme für die Tabelle F_BreastCancerQoL
F_BreastCancerQoL_df_noid = F_BreastCancerQoL_df.drop(columns=["Primary_Tumor_ID"])
F_BreastCancerQoL_hashes = hash_pandas_object(F_BreastCancerQoL_df_noid)
print("Checksumme F_BreastCancerQoL: ", F_BreastCancerQoL_hashes.sum())
if (F_BreastCancerQoL_hashes.sum() == -3476499510884993290):
    print("Die Überprüfung der pseudonymisierten Daten in F_BreastCancerQoL per Checksumme war erfolgreich.\n")
else:
    print("Die Überprüfung der pseudonymisierten Daten in F_BreastCancerQoL per Checksumme war nicht erfolgreich.\n")

# Bisherige Tabellen in der DB löschen
conn.execute("DROP TABLE IF EXISTS d_PrimaryTumor;")
conn.execute("DROP TABLE IF EXISTS F_BreastCancerQoL;")

# Veränderte, Anonymisierte Tabellen in die DB schreiben
d_primary_tumor_table_df.to_sql(name="d_PrimaryTumor", con=conn, if_exists='replace', index=False)
F_BreastCancerQoL_df.to_sql(name="F_BreastCancerQoL", con=conn, if_exists='replace', index=False)

# Zur Sicherheit: Alle Tabellennamen aus der Datenbank ziehen...
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tablelist = cursor.fetchall()

# ...und die Tabellennamen ausgeben
if tablelist == []:
    print("In der Data Warehouse Datenbank sind keine Tabellen vorhanden. Sie können mit der Ausführung des Skriptes fortfahren.")
else:
    print("Folgende Tabellen sind in der Datenbank vorhanden: ")
    print(tablelist)

# Testen, ob die gehashten Werte in der DB stehen
cursor.execute("SELECT ID from d_PrimaryTumor;")
hashtest_primary_tumor = cursor.fetchall()
cursor.execute("SELECT Primary_Tumor_ID from F_BreastCancerQoL;")
hashtest_encounters = cursor.fetchall()

print("\n" + "Hashtabellen von d_PrimaryTumor und F_BreastCancerQoL:")
print(hashtest_primary_tumor)
print(hashtest_encounters)

# Testen, ob die Übertragung mittels Dictionary funktioniert hat:
amount_ids = 0
for primary_tumor_id_total in hashtest_primary_tumor:
    if primary_tumor_id_total in hashtest_encounters:
        amount_ids = amount_ids + 1

# Erhält man als Ergebnis 5, so sollte der Vorgang funktioniert haben
if amount_ids == 5:
    print("\n" + "Die ID aller 5 Primary_Tumor_IDs in der Tabelle d_PrimaryTumor wurde gehasht.")

# Final überprüfen, ob vor und nach dem Hashvorgang die gleiche Anzahl an Ids in der Tabelle F_BreastCancerQoL stehen:
cursor.execute("SELECT 'Primary_Tumor_ID' FROM F_BreastCancerQoL;")
after_hash_primary_tumor_F_BreastCancerQoL = cursor.fetchall()
after_hash_primary_tumor_F_BreastCancerQoL = len(after_hash_primary_tumor_F_BreastCancerQoL)

if before_hash_primary_tumor_F_BreastCancerQoL == after_hash_primary_tumor_F_BreastCancerQoL:
    print("Alle Primary_Tumor_IDs in der Tabelle F_BreastCancerQoL wurden erfolgreich gehasht.")
else:
    print("Es wurden nicht alle Primary_Tumor_IDs in der F_BreastCancerQoL gehasht.")


Anzahl der Primary_Tumor_IDs (muss 11 sein): 5
Checksumme d_PrimaryTumor:  842637750972571563
Die Überprüfung der pseudonymisierten Daten in d_PrimaryTumor per Checksumme war erfolgreich.

Checksumme F_BreastCancerQoL:  -2919313348045796859
Die Überprüfung der pseudonymisierten Daten in F_BreastCancerQoL per Checksumme war nicht erfolgreich.

Folgende Tabellen sind in der Datenbank vorhanden: 
[('d_Agegroups',), ('d_Medications',), ('d_Cancertherapy',), ('d_Comorbidity',), ('d_Patients',), ('d_PrimaryTumor',), ('F_BreastCancerQoL',)]

Hashtabellen von d_PrimaryTumor und F_BreastCancerQoL:
[('9857e30fb700043fc6a3f65be4baeb7b681f0be13bff5cec3003c0fe663c29b5',), ('677a07fdb8d04c870a030f672f253e0936f7b7a20d9c99b1a3e3f5292db55fdc',), ('8e5009a41674917b58940452363388780581464d4cd3b31a7a349e1d971f0bd4',), ('77c45d1de4f3f797acb847c5b5ee3b50b6b76f07b8af678ca2f8014032ef44df',), ('0bd0e762c19c8d362b0a3ae060c666fabdd0953beef23477a448b6b326703879',)]
[(None,), ('0bd0e762c19c8d362b0a3ae060c666fabdd0


# **Commit der Änderungen und Schließen der Datenbankverbindung:**

Die durchgeführten Änderungen werden in der Datenbank bestätigt, und die Verbindung wird geschlossen.

In [138]:
conn.commit()
conn.close()

# **Abschluss und Nutzung der Data Warehouse-Datenbank:**


---



Nach erfolgreicher Ausführung dieses Colab-Notebooks befindet sich die erstellte Data Warehouse-Datenbank (`TeamB_Warehouse.db`) auf deinem Google Drive. Diese Datenbank bietet eine optimierte Struktur umfassende Analysen des bereitgestellten Synthea Breast Cancer-Datensatzes gemäß des [*Statistical Analysis Plans*](https://github.com/Fuenfgeld/DMA2024TeamB/wiki/Statistischer-Analyse-Plan). Die erstellten Tabellen enthalten aggregierte und transformierte Informationen, die besonders für komplexe Abfragen geeignet sind. Die Data Warehouse-Datenbank bietet somit eine solide Grundlage für weiterführende Analysen und Forschungsprojekte.
